In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install rioxarray

import numpy as np
import random
import os
import rasterio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

data = np.load("/content/drive/MyDrive/new_modis_data.npz")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 104.7 MB/s eta 0:00:00


# Training Data


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation, UpSampling2D,
    Concatenate, Layer, Dropout, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
)
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import EfficientNetB4, MobileNetV2, EfficientNetB0, MobileNetV3Large, VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from keras.metrics import MeanIoU
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
X_train = data["X_train"]
y_train = data["y_train"]

X_val = data["X_val"]
y_val = data["y_val"]

X_test = data["X_test"]
y_test = data["y_test"]

X_train.shape
data.close()

In [ ]:
input_shape = (32, 32, 3)

In [ ]:
recall = tf.keras.metrics.Recall(name='recall')
precision = tf.keras.metrics.Precision(name='precision')

## EfficientNetB0 - Ashley

In [ ]:
def create_efficientnetb0():
    base_model = EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    return model

In [ ]:
efficientnetb0 = create_efficientnetb0()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

efficientnetb0.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[recall, precision, 'accuracy']
)

efficientnetb0.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=20, callbacks=callbacks)

Epoch 1/20
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6575 - loss: 0.6514 - precision: 0.2939 - recall: 0.0094
Epoch 1: val_loss improved from inf to 0.63697, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 43s 33ms/step - accuracy: 0.6575 - loss: 0.6514 - precision: 0.2939 - recall: 0.0094 - val_accuracy: 0.6668 - val_loss: 0.6370 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/20
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6651 - loss: 0.6441 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2: val_loss improved from 0.63697 to 0.63651, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.6651 - loss: 0.6441 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.6668 - val_loss: 0.6365 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/20
690/693 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6703 - loss: 0.6383 - precision: 0.0000e

In [ ]:
best_efficientnetb0 = load_model("best_model.keras", custom_objects={
    'recall': recall,
    'precision': precision
})

best_efficientnetb0.evaluate(X_test, y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.6811 - loss: 0.6266 - precision: 0.0000e+00 - recall: 0.0000e+00


[0.6364513039588928, 0.0, 0.0, 0.6667569279670715]

## MobileNetV2 - Shirina

In [ ]:
def create_mobilenetv2():
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    return model

In [ ]:
mobilenetv2 = create_mobilenetv2()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

mobilenetv2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[recall, precision, 'accuracy']
)

mobilenetv2.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=50, callbacks=callbacks)

<ipython-input-10-7b699983fec9>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')


Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6719 - loss: 0.6056 - precision: 0.5175 - recall: 0.1285
Epoch 1: val_loss improved from inf to 0.51378, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - accuracy: 0.6720 - loss: 0.6056 - precision: 0.5178 - recall: 0.1287 - val_accuracy: 0.7545 - val_loss: 0.5138 - val_precision: 0.7051 - val_recall: 0.4525 - learning_rate: 0.0010
Epoch 2/50
687/693 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7568 - loss: 0.5145 - precision: 0.7108 - recall: 0.4569
Epoch 2: val_loss improved from 0.51378 to 0.49678, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7568 - loss: 0.5145 - precision: 0.7108 - recall: 0.4570 - val_accuracy: 0.7607 - val_loss: 0.4968 - val_precision: 0.7433 - val_recall: 0.4305 - learning_rate: 0.0010
Epoch 3/50
688/693 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7666 - loss: 0.4949 - precision: 0.7335 - recall: 0.4725
Epoch 3: val_loss

In [ ]:
best_mobilenetv2 = load_model("best_model.keras", custom_objects={
    'recall': recall,
    'precision': precision
})

best_mobilenetv2.evaluate(X_test, y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.8059 - loss: 0.4381 - precision: 0.7686 - recall: 0.5608


[0.4421641528606415, 0.5434606075286865, 0.77073734998703, 0.7939902544021606]

## SqueezeNet - Ashley

In [ ]:
def fire_module(x, squeeze, expand):
    x = layers.Conv2D(squeeze, (1,1), activation='relu', padding='same')(x)
    left = layers.Conv2D(expand, (1,1), activation='relu', padding='same')(x)
    right = layers.Conv2D(expand, (3,3), activation='relu', padding='same')(x)
    return layers.concatenate([left, right], axis=-1)

def create_squeezenet(input_shape=(32, 32, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(96, (7,7), strides=(2,2), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)
    x = fire_module(x, 16, 64)
    x = fire_module(x, 16, 64)
    x = fire_module(x, 32, 128)
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)
    x = fire_module(x, 32, 128)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=output)

    return model

In [ ]:
squeezenet = create_squeezenet()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

squeezenet.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[recall, precision, 'accuracy']
)

squeezenet.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=20, callbacks=callbacks)

Epoch 1/20
692/693 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7284 - loss: 0.5354 - precision: 0.8479 - recall: 0.6168
Epoch 1: val_loss improved from inf to 0.32407, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 95s 58ms/step - accuracy: 0.7286 - loss: 0.5352 - precision: 0.8478 - recall: 0.6169 - val_accuracy: 0.8714 - val_loss: 0.3241 - val_precision: 0.8987 - val_recall: 0.6921 - learning_rate: 0.0010
Epoch 2/20
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.8854 - loss: 0.3015 - precision: 0.8619 - recall: 0.7730
Epoch 2: val_loss improved from 0.32407 to 0.27172, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 37s 52ms/step - accuracy: 0.8854 - loss: 0.3015 - precision: 0.8620 - recall: 0.7730 - val_accuracy: 0.8971 - val_loss: 0.2717 - val_precision: 0.8782 - val_recall: 0.8026 - learning_rate: 0.0010
Epoch 3/20
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8956 - loss: 0.2716 - precision: 0.8935 - recall: 0.7834
Epoch 3: val

In [ ]:
best_squeezenet = load_model("best_model.keras", custom_objects={
    'recall': recall,
    'precision': precision
})

best_squeezenet.evaluate(X_test, y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9428 - loss: 0.1732 - precision: 0.9524 - recall: 0.8629


[0.16746650636196136,
 0.8740860819816589,
 0.9547471404075623,
 0.9442338943481445]

## VGGNet - Ashley

In [ ]:
def build_vgg(input_shape=(32, 32, 3)):
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return Model(inputs, outputs)


In [ ]:
vgg = build_vgg()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

vgg.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[recall, precision, 'accuracy']
)

vgg.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=50, callbacks=callbacks)

Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8269 - loss: 0.4163 - precision: 0.9017 - recall: 0.7320
Epoch 1: val_loss improved from inf to 0.24397, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.8270 - loss: 0.4162 - precision: 0.9017 - recall: 0.7320 - val_accuracy: 0.9136 - val_loss: 0.2440 - val_precision: 0.9262 - val_recall: 0.8050 - learning_rate: 0.0010
Epoch 2/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9169 - loss: 0.2291 - precision: 0.9118 - recall: 0.8296
Epoch 2: val_loss improved from 0.24397 to 0.21743, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9169 - loss: 0.2291 - precision: 0.9118 - recall: 0.8296 - val_accuracy: 0.9207 - val_loss: 0.2174 - val_precision: 0.9114 - val_recall: 0.8440 - learning_rate: 0.0010
Epoch 3/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9254 - loss: 0.2016 - precision: 0.9276 - recall: 0.8454
Epoch 3: val_los

In [ ]:
best_vgg = load_model("best_model.keras", custom_objects={
    'recall': recall,
    'precision': precision
})

best_vgg.evaluate(X_test, y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9501 - loss: 0.1571 - precision: 0.9593 - recall: 0.8800


[0.16228215396404266,
 0.8813972473144531,
 0.9576346278190613,
 0.9474824070930481]

## Simple CNN - Rebecca

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import Recall
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),

    layers.Dense(128, activation='relu'),

    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Recall(name='recall'), Precision(name='precision')])
# model.summary()


### monitor = recall

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_recall',mode='max',patience=5,restore_best_weights=True,verbose=1),
    ModelCheckpoint('best_model_recall.keras',monitor='val_recall',mode='max',save_best_only=True,verbose=1),
    ReduceLROnPlateau(monitor='val_recall',mode='max',factor=0.5,patience=3,min_lr=1e-6,verbose=1)
]
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7852 - loss: 0.4622 - precision: 0.6982 - recall: 0.5132
Epoch 1: val_recall improved from -inf to 0.77904, saving model to best_model_recall.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7852 - loss: 0.4621 - precision: 0.6984 - recall: 0.5134 - val_accuracy: 0.9180 - val_loss: 0.2413 - val_precision: 0.9687 - val_recall: 0.7790 - learning_rate: 0.0010
Epoch 2/50
692/693 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9274 - loss: 0.2042 - precision: 0.9363 - recall: 0.8405
Epoch 2: val_recall improved from 0.77904 to 0.80666, saving model to best_model_recall.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9274 - loss: 0.2043 - precision: 0.9363 - recall: 0.8405 - val_accuracy: 0.9299 - val_loss: 0.1987 - val_precision: 0.9793 - val_recall: 0.8067 - learning_rate: 0.0010
Epoch 3/50
686/693 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9381 - loss: 0.1725 - precision: 0.9574 - recall: 0.8542


In [ ]:
model = load_model('best_model_recall.keras')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Recall(name='recall'), Precision(name='precision')])
results = model.evaluate(X_test, y_test, batch_size=16, verbose=1)

231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9378 - loss: 0.1788 - precision: 0.8943 - recall: 0.9120


### monitor = loss

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss',mode='min',patience=5,restore_best_weights=True,verbose=1),
    ModelCheckpoint('best_model_loss.keras',monitor='val_loss',mode='min',save_best_only=True,verbose=1),
    ReduceLROnPlateau(monitor='val_loss',mode='min',factor=0.5,patience=3,min_lr=1e-6,verbose=1)
]
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7942 - loss: 0.4450 - precision: 0.7070 - recall: 0.5499
Epoch 1: val_loss improved from inf to 0.25726, saving model to best_model_loss.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7943 - loss: 0.4449 - precision: 0.7071 - recall: 0.5501 - val_accuracy: 0.8931 - val_loss: 0.2573 - val_precision: 0.8297 - val_recall: 0.8546 - learning_rate: 0.0010
Epoch 2/50
689/693 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9136 - loss: 0.2309 - precision: 0.9170 - recall: 0.8198
Epoch 2: val_loss improved from 0.25726 to 0.20250, saving model to best_model_loss.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9136 - loss: 0.2308 - precision: 0.9171 - recall: 0.8199 - val_accuracy: 0.9323 - val_loss: 0.2025 - val_precision: 0.9415 - val_recall: 0.8497 - learning_rate: 0.0010
Epoch 3/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9350 - loss: 0.1789 - precision: 0.9548 - recall: 0.8414
Epoch 3: 

In [ ]:
model = load_model('best_model_loss.keras')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Recall(name='recall'), Precision(name='precision')])
results = model.evaluate(X_test, y_test, batch_size=16, verbose=1)

231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9448 - loss: 0.1641 - precision: 0.9450 - recall: 0.8771


## InceptionV3 - Vicki

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, Resizing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
import tensorflow.keras.backend as K


def build_inception_model():
    inputs = Input(shape=(32, 32, 3))
    x = Resizing(128, 128)(inputs)
    base = InceptionV3(weights=None, include_top=False, input_tensor=x)
    x = GlobalAveragePooling2D()(base.output)
    output = Dense(1, activation='sigmoid')(x)
    return Model(inputs=inputs, outputs=output)

inception = build_inception_model()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

inception.compile(
    optimizer=Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

inception.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.7719 - loss: 0.5070 - precision: 0.6832 - recall: 0.6020
Epoch 1: val_loss improved from inf to 0.31389, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 165s 137ms/step - accuracy: 0.7720 - loss: 0.5069 - precision: 0.6833 - recall: 0.6020 - val_accuracy: 0.8776 - val_loss: 0.3139 - val_precision: 0.8438 - val_recall: 0.7766 - learning_rate: 1.0000e-04
Epoch 2/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.8608 - loss: 0.3467 - precision: 0.8247 - recall: 0.7241
Epoch 2: val_loss improved from 0.31389 to 0.27009, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 61s 66ms/step - accuracy: 0.8609 - loss: 0.3467 - precision: 0.8247 - recall: 0.7241 - val_accuracy: 0.9058 - val_loss: 0.2701 - val_precision: 0.9575 - val_recall: 0.7506 - learning_rate: 1.0000e-04
Epoch 3/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.8942 - loss: 0.2742 - precision: 0.8820 - recall: 0.7968
E

In [ ]:
best_inception = load_model("best_model.keras",
                        custom_objects={'Precision': Precision, 'Recall': Recall})

results = best_inception.evaluate(X_test, y_test, verbose=1)


116/116 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.9627 - loss: 0.1284 - precision: 0.9540 - recall: 0.9277
